## Start by setting week number
#### Wait for ESPN data update until at least Wednesday morning, but run latest by Thursday night

https://www.espn.com/nfl/stats/team/_/stat/total/season/2021/seasontype/2

In [1]:
SET_WEEK = 14
week_name = 'week_' + str(SET_WEEK)
print(week_name)

week_14


In [2]:
from scrapers.scrape_2021 import get_2021_data
import pandas as pd
import numpy as np
import pickle

import warnings
warnings.filterwarnings('ignore')

### Models

In [3]:
rf = '../modeling/rf.pkl'
gbm = '../modeling/gbm.pkl'
lgb = '../modeling/lgb.pkl'
catboost = '../modeling/catboost.pkl'

with open(rf, 'rb') as file:
    RF = pickle.load(file)
    
with open(gbm, 'rb') as file:
    GBM = pickle.load(file)
    
with open(lgb, 'rb') as file:
    LGB = pickle.load(file)
    
with open(catboost, 'rb') as file:
    CATBOOST= pickle.load(file)

### 2021 data

In [4]:
%%time

X = get_2021_data()

# from boruta elimination method:
keep_cols = ['offense_points_per_game', 'offense_passing_AVG', 'offense_passing_YDS/G', 'offense_passing_RTG', 
             'offense_receiving_AVG', 'offense_rushing_YDS/G', 'defense_points_per_game', 'defense_passing_AVG', 
             'defense_receiving_AVG', 'defense_rushing_YDS/G', 'defense_passing_SYL_per_game', 
             'offense_downs_Third Downs_ATT_per_game', 'defense_downs_Third Downs_ATT_per_game', 
             'defense_downs_Fourth Downs_ATT_per_game', 'defense_pass_to_rush_ratio', 
             'defense_downs_First Downs_penalty_ratio', 'offense_passing_TD_per_game', 'offense_pass_TD_to_INT']

X = X[['Team', 'games_played', 'season'] + keep_cols].copy()

X.head(2)

Wall time: 19.1 s


,Team,games_played,season,offense_points_per_game,offense_passing_AVG,offense_passing_YDS/G,offense_passing_RTG,offense_receiving_AVG,offense_rushing_YDS/G,defense_points_per_game,...,defense_receiving_AVG,defense_rushing_YDS/G,defense_passing_SYL_per_game,offense_downs_Third Downs_ATT_per_game,defense_downs_Third Downs_ATT_per_game,defense_downs_Fourth Downs_ATT_per_game,defense_pass_to_rush_ratio,defense_downs_First Downs_penalty_ratio,offense_passing_TD_per_game,offense_pass_TD_to_INT
0,Tampa Bay Buccaneers,13,2021,31.5,7.5,314.2,104.1,10.9,96.0,22.8,...,9.6,91.2,16.538462,12.615385,12.846154,1.461538,1.985075,0.111913,2.769231,3.600000
1,Dallas Cowboys,13,2021,29.2,7.6,280.6,98.1,11.2,128.5,22.1,...,12.4,110.2,18.615385,13.461538,12.461538,1.230769,1.471875,0.118577,2.000000,2.363636


In [5]:
X.fillna(0, inplace = True)
X.replace([np.inf, -np.inf], 0, inplace=True)

In [6]:
X['games_played'].unique().tolist()

[13]

In [7]:
# X[X['games_played'] == 8]

In [8]:
predictions = X[['Team', 'season']].copy()
predictions['week_marker'] = week_name

predictions['pred_rf'] = RF.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_gbm'] = GBM.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_lgb'] = LGB.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_catboost'] = CATBOOST.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_mean'] = (predictions['pred_rf'] + predictions['pred_gbm'] + predictions['pred_lgb'] + predictions['pred_catboost']) / 4

predictions.sort_values('pred_mean', ascending = False, inplace = True)
predictions.head(3)

,Team,season,week_marker,pred_rf,pred_gbm,pred_lgb,pred_catboost,pred_mean
6,Los Angeles Rams,2021,week_14,0.308492,0.241532,0.249515,0.302776,0.275579
0,Tampa Bay Buccaneers,2021,week_14,0.235300,0.182935,0.230662,0.283474,0.233093
8,Arizona Cardinals,2021,week_14,0.223487,0.144152,0.144527,0.226717,0.184721


In [9]:
predictions.to_csv('weekly_runs/' + week_name + '.csv', index = False)

### Check weekly run

In [10]:
predictions

,Team,season,week_marker,pred_rf,pred_gbm,pred_lgb,pred_catboost,pred_mean
6,Los Angeles Rams,2021,week_14,0.308492,0.241532,0.249515,0.302776,0.275579
0,Tampa Bay Buccaneers,2021,week_14,0.235300,0.182935,0.230662,0.283474,0.233093
8,Arizona Cardinals,2021,week_14,0.223487,0.144152,0.144527,0.226717,0.184721
17,New England Patriots,2021,week_14,0.160766,0.149496,0.125581,0.227272,0.165779
7,Buffalo Bills,2021,week_14,0.188460,0.098527,0.135958,0.221286,0.161058
1,Dallas Cowboys,2021,week_14,0.184269,0.088831,0.128438,0.191213,0.148188
9,Las Vegas Raiders,2021,week_14,0.123070,0.077115,0.120105,0.196571,0.129215
11,San Francisco 49ers,2021,week_14,0.138476,0.065721,0.096187,0.216200,0.129146
13,Cincinnati Bengals,2021,week_14,0.135667,0.068838,0.092312,0.184677,0.120373
12,Green Bay Packers,2021,week_14,0.110589,0.060368,0.101916,0.199804,0.118169
